# Importar librerias

In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from quickda.clean_data import *

# Limpiar los data y seleccionar las columnas a utilizar

In [52]:
data = pd.read_csv('baseball_reference_2016_scrape.csv')

data["attendance"] = data["attendance"].str.replace("']", "", regex=True)
data["attendance"] = data["attendance"].str.replace(",", "", regex=True)
data = data[data["attendance"].str.isdigit()]
data = data.astype({"attendance": int})

data['game_duration'] = data['game_duration'].str.replace(": ", "")
v = data["game_duration"].str.split(':', expand=True).astype(int)
data["game_duration"] = v[0]*60 +v[1]
data = data.astype({"attendance": int}, {'game_duration': int})

data["venue"] = data["venue"].str.replace(": ", "", regex=True)

data = data[["attendance", "home_team", "away_team", "game_duration", "start_time", "venue"]]

# Codificar los data

In [53]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

variables_a_codificar = ["home_team", "away_team", "venue", "start_time", "game_duration"]
transCol = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), variables_a_codificar)], remainder='passthrough')
X = np.array(transCol.fit_transform(X))

# Crear el modelo

In [54]:
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size = 0.2, random_state = 0)
regresor = LinearRegression()
regresor.fit(X_entreno, y_entreno)

LinearRegression()

# Cálculo de R cuadrado

In [55]:
r_squared = regresor.score(X_entreno, y_entreno)
print(r_squared)

0.7454042906456203


# Ecuación del modelo

In [56]:
coef = regresor.coef_
intercept = regresor.intercept_
print(coef, intercept)

[ 8.74726335e+15  8.72088128e+15 -1.12292606e+15 -9.44073424e+15
 -1.16674649e+17  5.54911001e+15  3.74875529e+16 -2.22600907e+16
  4.07766263e+16  9.38604884e+16  1.72500507e+17  6.69503594e+16
 -1.31581775e+16 -9.70752834e+15  3.16067868e+17 -3.05459277e+16
  5.72434105e+15  4.75773125e+17  6.72079475e+17  2.03571674e+17
  1.23973644e+16  8.82172065e+15 -3.81100443e+17 -4.61411809e+16
 -2.58222029e+17  3.93649004e+16 -6.03475122e+16  4.88284355e+16
  1.26337420e+17 -1.56209924e+17 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16 -5.87385000e+16
  1.16805924e+17  8.38229

# Predecir un resultado

In [57]:
varToPredict = []
    
homeTeam = "Kansas City Royals"
awayTeam = "New York Mets"
gameDuration = "193"
startTime = "Start Time: 7:38 p.m. Local"
venue = "Kauffman Stadium"

for column in transCol.get_feature_names_out():
    splitVal = column.replace("encoder__", "").split("_")
    valType = splitVal[0]
    testVal = splitVal[len(splitVal) - 1].strip()

    if venue == testVal and valType == "venue":
        varToPredict.append(1)
    elif homeTeam == testVal and valType == "home":
        varToPredict.append(1)
    elif awayTeam == testVal and valType == "away":
        varToPredict.append(1)
    elif gameDuration == testVal and valType == "game":
        varToPredict.append(1)
    elif startTime == testVal and valType == "start":
        varToPredict.append(1)
    else:
        varToPredict.append(0)

print(regresor.predict([varToPredict]))

[43040.]
